In [1]:
%%configure -f
{"name": "arik", "executorMemory": "1G", "numExecutors": 2, "executorCores": 1,
 "conf": {"spark.yarn.appMasterEnv.PYSPARK_PYTHON":"python3"}}

In [18]:
import os
import requests
import subprocess
from io import StringIO
from pyarrow import hdfs

def copy_file_to_hdfs(local, dest):
    cmd = '/opt/hadoop/bin/hdfs dfs -put {} {}'.format(local, dest)
    try:
        stdout = subprocess.check_output(
            cmd,
            stderr = subprocess.STDOUT,
            shell=True).decode('utf-8')
    except Exception as e:
        return (1, e.output)
    return (0, stdout)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
def download_to_hdfs(file_data):
    root = os.path.dirname(file_data[0])
    basename = os.path.basename(file_data[0])
    url, path = root.split('sas/')
    #dr = parts[4]
    #ver = parts[8]
    #name = parts[-1]
    #basename = '{}-{}-{}'.format(dr,ver,name)
    dest = 'hdfs:///manga/data/{0}'.format(basename)
    # cache a local copy of file to put to hdfs
    with open(basename, 'wb') as f:
        f.write(file_data[1])
    copy = copy_file_to_hdfs(basename, dest)
    os.remove(basename)
    return copy

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
path = 'https://data.sdss.org/sas/dr15/manga/spectro/redux/v2_4_3/8485/stack/manga-8485-1901-LOGRSS.fits.gz'
#path = '/sciserver/vc/manga/vc/sas/dr15/manga/spectro/redux/v2_4_3/8485/stack/manga-8485-1901-LOGRSS.fits.gz'
base = 'dr15/manga/spectro/redux/v2_4_3/8485/stack/manga-8485-1901-LOGRSS.fits.gz'
dest = 'hdfs:///manga/data/{}'.format(base)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#copy = copy_file_to_hdfs('manga-8485-1901-LOGRSS.fits.gz', dest)
#copy

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1, CalledProcessError())

In [6]:
#import glob
#files = glob.glob('/sciserver/vc/manga/vc/sas/dr15/manga/spectro/redux/v2_4_3/8485/stack/manga*LOG*')
files = [path]
files

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['https://data.sdss.org/sas/dr15/manga/spectro/redux/v2_4_3/8485/stack/manga-8485-1901-LOGRSS.fits.gz']

In [20]:
# to load across http
sc.parallelize(files).map(lambda x: (x, requests.get(x).content)).map(download_to_hdfs).collect()
# to load local sciserver
#sc.parallelize(files).map(download_to_hdfs).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1, b"put: `hdfs:///manga/data/manga-8485-1901-LOGRSS.fits.gz': File exists\n")]